In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.getActiveSession()

In [2]:
myRange = spark.range(1000).toDF("number")

In [3]:
divisBy2 = myRange.where("number % 2 = 0")

In [4]:
divisBy2.count()

500

In [8]:
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("./2015-summary.csv")

In [9]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [10]:
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#29 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#29 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=71]
      +- FileScan csv [DEST_COUNTRY_NAME#27,ORIGIN_COUNTRY_NAME#28,count#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/work-dir/workspace/pyspark/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [11]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [12]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#27, 200), ENSURE_REQUIREMENTS, [plan_id=87]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/work-dir/workspace/pyspark/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#27, 200), ENSURE_REQUIREMENTS, [plan_id=100]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIn

In [13]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [14]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [15]:
from pyspark.sql.functions import desc

flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [16]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#112L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#27,destination_total#112L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[sum(count#29)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#27, 200), ENSURE_REQUIREMENTS, [plan_id=238]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[partial_sum(count#29)])
            +- FileScan csv [DEST_COUNTRY_NAME#27,count#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/work-dir/workspace/pyspark/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


